In [1]:
from datasets import load_dataset
dataset = load_dataset("openai_humaneval")

In [2]:
infos = dataset["test"]
function_defs = infos["prompt"]
function_sols = infos["canonical_solution"]

In [3]:
full_code_list = [""]*len(function_defs)
for i, (function_def, function_sol) in enumerate(zip(function_defs, function_sols)):
    full_function = function_def.split('    """')[0] + function_sol
    full_code_list[i] = full_function

In [4]:
from openai_code import explain_code, generate_inputs, generate_unittest_advanced

import yaml

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
openai_key = config["openai_key"]

In [12]:
code = full_code_list[1]
print(code)

from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result



In [13]:
code_explanation = explain_code(code, openai_key)
input_list = generate_inputs(code, code_explanation, openai_key)

In [14]:
print(eval(input_list))

['(a(b)c)', '(a(b(c)d)e)', '(a(b(c)d)e(f)g)', '(a)', '(a(b))']


In [22]:
def execute_function_from_string(func_string, *args, **kwargs):
    # Execute the function definition in the current scope
    exec(func_string)

    # Get the function from locals    
    start_index = func_string.find("def ")
    end_index = func_string.find("(")
    if start_index == -1 or end_index == -1:
        raise ValueError("No function definition found in the provided string.")

    # Extract the function definition substring
    function_definition = func_string[start_index:end_index]
    function_name = function_definition.split("(")[0][4:]
    function_to_call = locals().get(function_name)

    # Check if the function exists
    if function_to_call is not None:
        # Call the function with arguments and return the result
        return function_to_call(*args, **kwargs)
    else:
        raise ValueError("No function named 'function' found in the provided string.")

In [23]:
output_list = [0]*len(eval(input_list))
for i, input in enumerate(eval(input_list)):
    output_list[i] = execute_function_from_string(code, *input)

separate_paren_groups


TypeError: separate_paren_groups() takes 1 positional argument but 7 were given

In [17]:
print(output_list)

[0, 0, 0, 0, 0]


In [11]:
unittest = generate_unittest_advanced(code, code_explanation, input_list, output_list, openai_key)
print(unittest)

import unittest

class TestHasCloseElements(unittest.TestCase):

    def test_has_close_elements_false(self):
        self.assertEqual(has_close_elements([1.0, 2.0, 3.0], 0.5), False)
        self.assertEqual(has_close_elements([0.1, 0.2, 0.3, 0.4], 0.05), False)
        self.assertEqual(has_close_elements([100.0, 105.0, 110.0], 1.0), False)

    def test_has_close_elements_true(self):
        self.assertEqual(has_close_elements([10.5, 15.2, 20.7], 5.0), True)
        self.assertEqual(has_close_elements([-5.0, -3.0, 0.0, 2.0], 2.5), True)

if __name__ == '__main__':
    unittest.main()
